In [5]:
import json

# Replace 'your_input_file.json' and 'your_output_file.jsonl' with the actual paths
input_file_path = 'conversations.json'
output_file_path = 'conversations.jsonl'

def convert_to_json_lines(input_file, output_file):
    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            json_data = json.load(infile)
            for line in json_data:
                json_line = json.dumps(line)
                outfile.write(json_line + '\n')
    except FileNotFoundError:
        print(f"The file '{input_file}' was not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

convert_to_json_lines(input_file_path, output_file_path)


In [10]:
import jsonlines

# Specify the path to the input jsonl file
input_file = 'conversations.jsonl'

# Specify the path to the output jsonl file
output_file = 'conversations_empty.jsonl'

# Open the input and output files
with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
    # Iterate over each entry in the input file
    for entry in reader:
        # Iterate over each message in the mapping
        for message_id, message in entry['mapping'].items():
            # Check if the message field is not None
            if message['message'] is not None:
                # Delete the content of the message
                message['message']['content']['parts'] = []
        
        # Write the modified entry to the output file
        writer.write(entry)